In [1]:
import numpy as np
import torch

import scipy.io
import mat73

import sparse

from tqdm.notebook import tqdm
from scipy.sparse.linalg import norm

In [5]:
# /data/zqazi004/decomposition_factors/reddit_tensor_indices_cp_factors_rankXXX.mat
path = input('Enter factors file path: ')
try:
    factors = scipy.io.loadmat(path)['c_factors']
except:
    factors = mat73.loadmat(path)['c_factors']

In [6]:
A, B = factors[1][0], factors[2][0]

print(A.shape, B.shape)

(65119, 10) (65119, 10)


In [7]:
data = scipy.io.loadmat('tensor_data/reddit_tensor_datas.mat')

In [8]:
# Extract the indices and values from the dictionary
indices = data['indices']
values = data['values'][0]
N = data['size'][0][0]

sparse_tensor = sparse.COO(torch.tensor(list(zip(*indices))), data=torch.tensor(values))

In [9]:
# Iterate through each slice along the first dimension of the sparse tensor
norms = []
for i in tqdm(range(1)):
    s_i = sparse_tensor[i]

    # Convert the sparse slice s to a dense numpy array
    projection = sparse.tensordot(sparse.tensordot(A.T, s_i, axes=1), B, axes=1)

    reconstruction = sparse.tensordot(sparse.tensordot(A, projection, axes=1), B.T, axes=1)

    norm_diff = np.linalg.norm(s_i - reconstruction, ord='fro')

    norms.append(norm_diff)

frobenius_norms_array = np.array(norms)

  0%|          | 0/3 [00:00<?, ?it/s]

AttributeError: module 'sparse' has no attribute 'sum'